# weekly_earnings

Send a request to CityBikes for the city of your choice. 

In [2]:
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [79]:
with open("weekly_earnings.json", "r") as read_file:
    result = json.load(read_file)

In [128]:
#read weekly earnings from json file and consolidate into csv format to act as data source
#for earnings vs housing prices trend comparison.
list_of_columns = []
dict_of_columns = result['meta']['view']['columns']
for x in dict_of_columns:
    list_of_columns.append(x['name'])

list_of_values = result['data']
df = pd.DataFrame(list_of_values, columns =list_of_columns)

df.drop(['sid', 'id', 'position', 'created_at', 'created_meta', 
        'updated_at', 'updated_meta', 'meta'], axis = 1, inplace = True)
df['Year'] = df['When'].str[0:4]

df.to_csv('weekly_earnings_yr.csv', index=False)

In [220]:
#read consumer index information and consolidate average consumer and housing indices together
#in a single csv file to act as data source for linear regression check.
df = pd.read_csv('consumer_index.csv')

df['Year'] = df['date'].str[0:4]

df['Year'] = df['Year'].convert_dtypes(int)

df = df[df['unit'] == 'Index, 2002=100']

lst_YR = df['Year'].unique().tolist()

df_temp = pd.DataFrame() 

lst_MEAN = []

for x in lst_YR:
    df_temp = df[df['Year'] == x]
    lst_MEAN.append(round(df_temp['index_value'].mean(),2))

lst_YR = [int(i) for i in lst_YR]

df_REP = pd.read_csv('real_estate_prices_avg.csv')

lst_REP = []

lst_REP = df_REP['Composite_HPI']

i = 0

while i < 21:
    lst_YR.pop(0)
    lst_MEAN.pop(0)
    i += 1
    
df_AVG = pd.DataFrame(
        {'Year': lst_YR,
         'avg_consumer_index': lst_MEAN,
         'Composite_HPI': lst_REP
        })

df_AVG.to_csv('consumer_index_&_HPI.csv', index=False)

In [26]:
#read real estate price information and consolidate averages on country and district level 
#and on yearly basis as data source for housing index and pricing analysis.
xlsx = pd.ExcelFile('real_estate_prices.xlsx')

dict_df_read = {}
dict_df_write = {}

for sheet_name in xlsx.sheet_names:
    dict_df_read[sheet_name] = pd.read_excel('real_estate_prices.xlsx', sheet_name=sheet_name)
    dict_df_read[sheet_name]['Year'] =dict_df_read[sheet_name]['Date'].dt.year
    lst_YR = dict_df_read[sheet_name]['Year'].unique().tolist()
    df_temp = pd.DataFrame() 
    df_sheet = pd.DataFrame()
    for x in lst_YR:
        df_temp = dict_df_read[sheet_name][dict_df_read[sheet_name]['Year'] == x]
        df_sheet = df_sheet.append(df_temp.mean(), ignore_index=True)
        
    dict_df_write[sheet_name] = df_sheet
    dict_df_write[sheet_name] = dict_df_write[sheet_name].round(2)
    dict_df_write[sheet_name]['Year'] = dict_df_write[sheet_name]['Year'].astype(int)
    
with pd.ExcelWriter('real_estate_prices_avg.xlsx') as writer:
    for sheet_name in xlsx.sheet_names:
        dict_df_write[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)